In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/54.4 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.7 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-vz82hncm
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-vz82hncm
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 M

In [2]:
from PIL import Image
import torch
from torch import nn, optim
import glob
import os
import pandas as pd
import json
import numpy as np
import clip
from torch.utils.data import Dataset, DataLoader, BatchSampler
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import random
from matplotlib.pyplot import imshow
import torchtext
import nltk, re, string, collections
from nltk.util import ngrams
import collections
%matplotlib inline
BATCH_SIZE =128
EPOCH = 5

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
 #!unzip "/content/drive/MyDrive/Copy of archive.zip" -d "/content/meme-project-raw"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

image = preprocess(Image.open("/content/meme-project-raw/beatlemaniabear.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["a diagram", "a dog", "a cat"]).to(device)
image.shape, text.shape

(torch.Size([1, 3, 224, 224]), torch.Size([3, 77]))

In [14]:
class MemeDataset(Dataset):
    def __init__(self, data, preprocess):
        self.preprocess = preprocess

        self.img_paths = []
        self.captions = []
        for img_path, captions in data.items():

            for cap in captions:
                #print('cap',cap)
                self.img_paths.append(img_path)
                self.captions.append(cap)

        self.processed_cache = {}
        for img_path in data:
            self.processed_cache[img_path] = self.preprocess(Image.open(img_path))
        self.img_paths_set = list(data.keys())
        self.path2label = {path: self.img_paths_set.index(path) for path in self.img_paths_set}

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        image = self.processed_cache[img_path]
        caption = self.captions[idx]
        label = self.path2label[img_path]
        return image, caption, label

train_dataset = MemeDataset(d_train, preprocess)
test_dataset = MemeDataset(d_test, preprocess)
len(train_dataset), len(test_dataset), train_dataset[0]

(239223,
 59514,
 (tensor([[[-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           ...,
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959]],
  
          [[-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           ...,
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418]],
  
          [[-0.9967, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9

In [ ]:
len(dataset),dataset[0]

NameError: name 'dataset' is not defined

In [15]:
len(train_dataset), len(test_dataset), train_dataset[0]

(239223,
 59514,
 (tensor([[[-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           ...,
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959],
           [-1.2959, -1.2959, -1.2959,  ..., -1.2959, -1.2959, -1.2959]],
  
          [[-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           ...,
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418],
           [-1.2418, -1.2418, -1.2418,  ..., -1.2418, -1.2418, -1.2418]],
  
          [[-0.9967, -0.9967, -0.9967,  ..., -0.9967, -0.9967, -0.9

In [16]:
i = 0
for k,v in train_dataset.path2label.items():
    i+=1
    print(k,v)
    if i == 10:
        break

/content/meme-project-raw/bill-o-reilly-fuck-it.jpg 0
/content/meme-project-raw/tolyatti.jpg 1
/content/meme-project-raw/teacher-of-physics.jpg 2
/content/meme-project-raw/me-iria-demasiado.jpg 3
/content/meme-project-raw/mormons-meme.jpg 4
/content/meme-project-raw/oh-so-youre.jpg 5
/content/meme-project-raw/pasqualebolado2.jpg 6
/content/meme-project-raw/soykimschmitz.jpg 7
/content/meme-project-raw/typical-antidno.jpg 8
/content/meme-project-raw/fuck-yeah-a-gents-parrot.jpg 9


In [17]:
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples):
        self.labels = labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size

train_labels = torch.tensor([item[2] for item in train_dataset])
train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)

test_labels = torch.tensor([item[2] for item in test_dataset])
test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)

# train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False)
# len(train_dataset), len(test_dataset), train_dataset[0]

In [18]:
for i, item in enumerate(train_sampler):
#     print(item)
#     print(len(item))
    labels = []
    for idx in item:
        label = train_dataset[idx][2]
        labels.append(label)
    break
len(labels), len(set(labels))

(256, 256)

In [19]:
for batch in train_dataloader:
    imgs, txts, labels = batch
    print(imgs.shape)
    print(len(txts))
    print(labels)
    print(labels.shape)
    print(torch.unique(labels).shape)
    break

torch.Size([256, 3, 224, 224])
256
tensor([ 168,  904,  244, 1442, 1985, 2216, 2061, 1110, 1437,  316, 2127, 1484,
        1395,  965, 1657, 1943, 1853, 1877,  991,  786,  661,  426,  952, 2073,
         551, 2258,  831, 2232,   19, 1949,   27, 2170,  378, 2194,  646,  543,
        1957,  915, 1137, 1713, 2159,  421,  486, 1162, 1284, 2273, 1811, 1206,
        1178, 2355,  480, 1548,  478,  296,  384, 1183, 2200,  128,  832,  796,
         458,  966, 1783,   97,  172, 1029, 1279,  472,  972,  548, 1923, 1505,
         389, 1931, 1258,  349,  628,  860, 1040, 1687,  524, 1530,   38,   70,
        2197, 1096,  468,  531, 2256,   50,  493, 1510, 1870,  416, 1036,  520,
          13,   18,  632,  896,  405, 1502, 1802,  977, 2180,  202,  101,  292,
        1925, 1584,  720, 1477, 1161,  738, 1435, 2088, 1297, 1852, 2182, 1144,
        1300, 2044, 1875,  692, 1658, 1207,  512, 1527,  506, 1131,  129, 1114,
        1901, 2099, 2137, 1150,  470, 2238,  497, 1351,  435, 1249, 1452,  679,
     

In [20]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [21]:
best_te_loss = 1e5
best_ep = -1

for epoch in range(EPOCH):
    print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)
#         print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step

    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step

    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
        torch.save(model.state_dict(), "best_model.pt")
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model.pt")

running epoch 0, best test loss 100000.0 after epoch -1


  0%|          | 0/934 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [23]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of best_model.pt"))
NUM_NEG = 127
NUM_TEST = 1000

In [24]:
n_correct = 0
for i in tqdm(range(NUM_TEST)):
    empty = True
    while empty:
        img_path = random.choice(list(d_test.keys()))
        image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
        name = img_path.split('/')[-1].split('.')[0]
        caps = d_test[img_path]
        if len(caps) > 0:
            pos_txt = random.choice(caps)
        #         pos_txt = ' '.join(pos_txt)
            empty = False
#     print(pos_txt)
    neg_i = 0
    neg_txts = []
    while neg_i < NUM_NEG:
        img_path = random.choice(list(d_test.keys()))
        neg_name = img_path.split('/')[-1].split('.')[0]
        if neg_name == name:
            continue
        caps = d_test[img_path]
        if len(caps) == 0:
            continue
        neg_txt = random.choice(caps)
        if neg_txt in neg_txts:
            continue
        neg_txts.append(neg_txt)
        neg_i += 1
#     print(name)
#     print(f"Positive caption: {pos_txt}")
#     print(f"Negative caption: {neg_txts}")
    text = clip.tokenize([pos_txt]+neg_txts).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#     print("Label probs:", probs)
#     print(np.argmax(probs))
    if np.argmax(probs) == 0:
        n_correct +=1
print(f"Test precision {n_correct/NUM_TEST}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Test precision 0.237


In [25]:
def sample1Caption(img_path, corpus, model, num_cand):
    image = preprocess(Image.open(img_path)).unsqueeze(0).to(device)
    i = 0
    txts = []
    while i < num_cand:
        txt = random.choice(corpus)
        if txt in txts:
            continue
        if len(txt.split())<5 or len(txt)>72:
            continue
        txts.append(txt)
        i += 1
    #     print(name)
    #     print(f"Positive caption: {pos_txt}")
    #     print(f"Negative caption: {neg_txts}")
    text = clip.tokenize(txts).to(device)

    with torch.no_grad():
        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    #     print("Label probs:", probs)
    #     print(np.argmax(probs))
    #     imshow(np.asarray(Image.open(img_path)))
    return txts[np.argmax(probs)]

In [27]:
model.load_state_dict(torch.load("/content/drive/MyDrive/Copy of best_model.pt"))
corpus = []
for txtlist in d_train.values():
    corpus += txtlist
len(corpus), corpus[0]

(239223, "FUCK IT I'M BUYING A NEW CAR BATTERY")

In [28]:
captions = {}
for img_path in tqdm(d_test.keys()):
    caption = sample1Caption(img_path, corpus, model, 1000)
    captions[img_path] = caption

  0%|          | 0/600 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 302.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 193.06 MiB is free. Process 2537 has 14.56 GiB memory in use. Of the allocated memory 13.40 GiB is allocated by PyTorch, and 1.01 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
for get_bleu in range(1,4):
    bleu_x_lst = []
    bleu_y_lst = []
    for p, caps in d_test.items():
        if not caps:
            continue
        bleu_x_lst.append(captions[p].split())
        splittedcaps = [x.split() for x in caps]
        bleu_y_lst.append(splittedcaps)
    BLEU = torchtext.data.metrics.bleu_score(bleu_x_lst, bleu_y_lst, max_n=get_bleu, weights=[1/get_bleu]*get_bleu)
    print(f"{get_bleu}-gram BLEU score: {BLEU}")

In [ ]:
sentences = list(captions.values())
BigramCtr = collections.Counter()
UnigramCtr = collections.Counter()
for sentence in sentences:
    BigramCtr.update(nltk.ngrams(sentence, 2))
    UnigramCtr.update(nltk.ngrams(sentence, 1))
# print("Unigram count:",len(BigramCtr)/len(sentences))
# print("Bigram count:",len(UnigramCtr)/len(sentences))
print("Unigram count:",len(BigramCtr))
print("Bigram count:",len(UnigramCtr))

In [ ]:
seen_path = random.choice(list(d_train.keys()))
pred_cap_seen = sample1Caption(seen_path, corpus, model, 1000)
gt_cap_seen = d_train[seen_path][:5]
imshow(Image.open(seen_path))
print(f"Some ground truth captions for this seen image: {gt_cap_seen}")
print(f"Caption sampled by fintuned CLIP for this seen image: {pred_cap_seen}")

In [ ]:
#new fine tune our model

In [4]:
#!unzip "/content/drive/MyDrive/mamidata/Copy of archive.zip" -d "/content/data"

In [6]:
# Define the paths to your dataset folder and CSV file
dataset_folder = "/content/data/MAMI_2022_images/training_images"
csv_file = "/content/drive/MyDrive/mamidata/Copy of MAMI_train_set.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

# Define a custom dataset class for MAMI dataset
class MAMIDataset(Dataset):
    def __init__(self, csv_file, root_dir,preprocess ):
        self.preprocess=preprocess
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = self.preprocess(Image.open(img_path))
        #image = Image.open(img_name)
        label = self.data.iloc[idx, 1]  # Assuming label is in the second column
        text = self.data.iloc[idx, 2][:77]  # Assuming text is in the third colum
        return image, text,label
dataset = MAMIDataset(csv_file,dataset_folder,preprocess)
#test_dataset = MemeDataset(d_test, preprocess)
#len(train_dataset), len(test_dataset), train_dataset[0]

In [7]:
#train_dataset1, test_dataset1 = train_test_split(dataset, test_size=0.2, random_state=42)
torch.manual_seed(123)
train_dataset,test_dataset = torch.utils.data.random_split(dataset,[9000,941])
train_dataset,test_dataset = torch.utils.data.random_split(train_dataset,[7000,2000])

In [11]:
# https://github.com/pytorch/pytorch/blob/e5742494f6080c8e6f43c37689fc18a7c4b39dfd/torch/utils/data/dataloader.py#L145
class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, labels, n_classes, n_samples):
        self.labels = labels
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}
        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.n_dataset = len(self.labels)
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < self.n_dataset:
            classes = np.random.choice(self.labels_set, self.n_classes, replace=True)
            indices = []
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return self.n_dataset // self.batch_size

train_labels = torch.tensor([item[2] for item in train_dataset])
train_sampler = BalancedBatchSampler(train_labels, BATCH_SIZE, 1)
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)

test_labels = torch.tensor([item[2] for item in test_dataset])
test_sampler = BalancedBatchSampler(test_labels, BATCH_SIZE, 1)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)

# train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False)
# len(train_dataset), len(test_dataset), train_dataset[0]

In [12]:
for i, item in enumerate(train_sampler):
#     print(item)
#     print(len(item))
    labels = []
    for idx in item:
        label = train_dataset[idx][2]
        labels.append(label)
    break
len(labels), len(set(labels))

(128, 2)

In [15]:
for batch in train_dataloader:
    imgs, txts, labels = batch
    print('imgs, txts, labels',imgs, txts, labels)
    print(imgs.shape)
    print(len(txts))
    print(labels)
    print(labels.shape)
    print(torch.unique(labels).shape)
    break

imgs, txts, labels tensor([[[[-1.4565e+00, -9.0176e-01, -8.4336e-01,  ...,  1.2150e+00,
            1.2296e+00,  1.1858e+00],
          [-1.4711e+00, -9.1636e-01, -8.8716e-01,  ...,  1.2296e+00,
            1.3026e+00,  1.2734e+00],
          [-1.4857e+00, -9.6015e-01, -9.3096e-01,  ...,  8.2086e-01,
            9.6684e-01,  1.0398e+00],
          ...,
          [-8.5796e-01,  2.3692e-01,  4.2670e-01,  ...,  9.2304e-01,
            9.6684e-01,  8.2086e-01],
          [-8.7256e-01,  2.0772e-01,  4.2670e-01,  ...,  9.3764e-01,
            9.2304e-01,  8.0626e-01],
          [-8.8716e-01,  2.0772e-01,  4.2670e-01,  ...,  9.0845e-01,
            9.2304e-01,  8.2086e-01]],

         [[-1.3619e+00, -8.0661e-01, -7.3157e-01,  ...,  5.5910e-01,
            6.0412e-01,  5.4409e-01],
          [-1.3769e+00, -8.0661e-01, -7.3157e-01,  ...,  2.5894e-01,
            3.6400e-01,  3.7901e-01],
          [-1.3769e+00, -8.3662e-01, -7.9160e-01,  ..., -5.5148e-01,
           -3.7138e-01, -2.6633e-01],
 

In [16]:
#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

if device == "cpu":
    model.float()

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader)*EPOCH)

In [19]:
len(train_dataloader)

54

In [20]:
best_te_loss = 1e5
best_ep = -1
for epoch in range(EPOCH):
    print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
    step = 0
    tr_loss = 0
    model.train()
    pbar = tqdm(train_dataloader, leave=False)
    for batch in pbar:
        step += 1
        optimizer.zero_grad()

        images, texts, _ = batch
        images = images.to(device)
        texts = clip.tokenize(texts).to(device)

#         print(images.shape, texts.shape)
        logits_per_image, logits_per_text = model(images, texts)

        ground_truth = torch.arange(BATCH_SIZE).to(device)

        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        tr_loss += total_loss.item()
        if device == "cpu":
            optimizer.step()
            scheduler.step()
        else:
            convert_models_to_fp32(model)
            optimizer.step()
            scheduler.step()
            clip.model.convert_weights(model)
        pbar.set_description(f"train batchCE: {total_loss.item()}", refresh=True)
    tr_loss /= step

    step = 0
    te_loss = 0
    with torch.no_grad():
        model.eval()
        test_pbar = tqdm(test_dataloader, leave=False)
        for batch in test_pbar:
            step += 1
            images, texts, _ = batch
            images = images.to(device)
            texts = clip.tokenize(texts).to(device)
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(BATCH_SIZE).to(device)

            total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
            te_loss += total_loss.item()
            test_pbar.set_description(f"test batchCE: {total_loss.item()}", refresh=True)
        te_loss /= step

    if te_loss < best_te_loss:
        best_te_loss = te_loss
        best_ep = epoch
        torch.save(model.state_dict(), "best_model1.pt")
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "last_model1.pt")

running epoch 0, best test loss 100000.0 after epoch -1


  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

epoch 0, tr_loss 0.32682743778935186, te_loss 0.5711263020833334
running epoch 1, best test loss 0.5711263020833334 after epoch 0


  0%|          | 0/54 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

epoch 3, tr_loss 0.05184653953269676, te_loss 0.615771484375
running epoch 4, best test loss 0.5711263020833334 after epoch 0


  0%|          | 0/54 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

epoch 4, tr_loss 0.05916143346715857, te_loss 0.5752766927083334


In [22]:
cp '/content/last_model1.pt' -d "/content/drive/MyDrive/ckeck"